# Rendering Prep

This notebook is for prototyping/prepping the driver for rendering the freespace dynamics dataset

NOTE: This was converted to a script and is run via command line.

In [1]:
import os
import sys
from pathlib import Path
import pickle

import bpy
import mathutils
import numpy as np

# Imports from this repository
sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.cloth_3d_util.util import loadInfo
from simulation.pipeline.simulate_garment_hanging_rest_state import \
    simulate_garment_hanging_rest_state
from simulation.blender_util_dylan.physics import set_sim_output_as_default_mesh_shape
from simulation.blender_util_dylan.checkpointer import BlendFileCheckpointer
from simulation.blender_util_dylan.gripper import GripperAnimation
from simulation.blender_util_dylan.debug import print_obj_keyframe_coordinates

from simulation.blender_util_dylan.render import enable_gpu_renders, render_dylan
from simulation.blender_util_dylan.debug import print_nested_dict_types
from simulation.blender_util.camera import generate_intrinsic
from simulation.cloth_3d_util.accessors.access_functions import (
    get_info, get_garment_texture, get_garment_metadata 
)

%load_ext autoreload
%autoreload 2

WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  retur

In [2]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "full_dataset_attempt_2"
GARMENTNETS_SAMPLE_DATASET_ZARR = (FILE_ROOT / ".." / ".." / "data" / 
                                   "garmentnets_simulation_dataset_sample.zarr")
assert (GARMENTNETS_SAMPLE_DATASET_ZARR.exists())
GARMENTNETS_SAMPLE_PATH = ["Tshirt", "samples"]

NUM_ACTION_SEQUENCES = 5
DYNAMICS_ANIMATION_DIR_TEMPLATE = "dynamics_seq_{idx}"
DYNAMICS_ANIMATION_FILENAME = "dynamics_animation.blend"

PLANE_OFFSET = 0.025  # [m]

## First, Render The Hanging Resting State (GarmentNets Input)

In [7]:
num_camera_angles = 4
camera_intrinsic = generate_intrinsic(1024, 1024, 2048)

for sample_dir in OUTPUT_ROOT.iterdir():
    sample_key = sample_dir.name

    render_output_example_1 = sample_dir / "rgb_0.png"
    render_output_example_2 = sample_dir / "uviz_3.exr"
    if render_output_example_1.exists() and render_output_example_2.exists():
        print(f"Already rendered hanging state for '{sample_key}', skipping render.")
        continue

    print(f"Animating {sample_key} resting state (GarmentNets input)")

    sample_id, garment_name, grip_vertex_idx_key = sample_key.split('_')
    
    # Load the initial sim results
    results_dict_filepath = sample_dir / "hanging_rest_state_results.pkl"
    assert(results_dict_filepath.exists())
    results_dict_smpl = pickle.load(results_dict_filepath.open('rb'))
    # print(results)
    cloth_state = results_dict_smpl["cloth_state"]

    assert (int(grip_vertex_idx_key) == results_dict_smpl["grip_vertex_idx"])

    # Load the garment information dictionary.
    accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
    sample_data = accessor.get_sample_data(sample_id, garment_name)
    print_nested_dict_types(sample_data)


    # Load the hanging rest state blend file.
    blend_path = sample_dir / "hanging_rest_state.blend"
    assert(blend_path.exists())
    bpy.ops.wm.open_mainfile(filepath=blend_path.as_posix())

    # Add the world material I accidentally deleted when simulating hanging rest state
    # bpy.ops.world.new()
    # print(bpy.data.worlds.keys())


    info = get_info(CLOTH3D_PATH, sample_id)
    garment_texture = get_garment_texture(CLOTH3D_PATH, sample_id, garment_name, info=info)
    garment_meta = get_garment_metadata(CLOTH3D_PATH, sample_id, garment_name, info=info)
    gender = garment_meta['gender']
    fabric = garment_meta['garment_fabric']

    enable_gpu_renders()
    render_dylan(
        output_path=sample_dir, 
        sample_id=sample_id, 
        garment_name=garment_name, 
        gender=gender, 
        fabric=fabric, 
        garment_verts=cloth_state["verts"], 
        garment_faces=cloth_state["faces"],
        garment_uv_verts=cloth_state["uv_verts"], 
        garment_uv_faces=cloth_state["uv_faces"], 
        garment_texture=garment_texture, 
        num_camera_angles=num_camera_angles, 
        camera_intrinsic=camera_intrinsic,
        z_offset=-0.4
    )

Already rendered hanging state for '00471_Tshirt_153', skipping render.
Animating 07135_Tshirt_4909 resting state (GarmentNets input)
human_verts <class 'numpy.ndarray'>
human_faces <class 'numpy.ndarray'>
human_gender <class 'int'>
garment_verts <class 'numpy.ndarray'>
garment_faces <class 'list'>
garment_uv_verts <class 'numpy.ndarray'>
garment_uv_faces <class 'list'>
garment_fabric <class 'str'>
garment_texture <class 'numpy.ndarray'>
Read blend: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/full_dataset_attempt_2/07135_Tshirt_4909/hanging_rest_state.blend
CUDA_NVIDIA GeForce RTX 2080 Ti_0000:01:00 True CUDA
CPU False CPU
CUDA_NVIDIA GeForce RTX 2080 Ti_0000:01:00_OptiX False OPTIX
Setting render configuration and printing failed sets.
	denoiser
	sampling_pattern
Fra:0 Mem:261.86M (Peak 309.88M) | Time:00:00.27 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | cloth
Fra:0 Mem:322.16M (Peak 341.98M)

KeyboardInterrupt: 

In [7]:
# Iterate through the action sequences first so we still get good sample coverage even if we have to
# stop the rendering early to train
for seq_idx in range(NUM_ACTION_SEQUENCES):
    for sample_dir in OUTPUT_ROOT.iterdir():
        sample_key = sample_dir.name
        print(f"Animating {sample_key}, action sequence: {seq_idx}")

        full_animation_blend_path = (sample_dir 
                                     / DYNAMICS_ANIMATION_DIR_TEMPLATE.format(idx=seq_idx) 
                                     / DYNAMICS_ANIMATION_FILENAME)
        
        assert(full_animation_blend_path.exists())

        sample_id, garment_name, grip_vertex_idx_key = sample_key.split('_')
        
        # Load the initial sim results
        results_dict_filepath = sample_dir / "hanging_rest_state_results.pkl"
        assert(results_dict_filepath.exists())
        results_dict_smpl = pickle.load(results_dict_filepath.open('rb'))
        # print(results)
        cloth_state = results_dict_smpl["cloth_state"]

        # Load the garment information dictionary.
        accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
        sample_data = accessor.get_sample_data(sample_id, garment_name)

        # Load the blend file.
        bpy.ops.wm.open_mainfile(filepath=full_animation_blend_path.as_posix())

        info = get_info(CLOTH3D_PATH, sample_id)
        garment_texture = get_garment_texture(CLOTH3D_PATH, sample_id, garment_name, info=info)
        garment_meta = get_garment_metadata(CLOTH3D_PATH, sample_id, garment_name, info=info)
        gender = garment_meta['gender']
        fabric = garment_meta['garment_fabric']

        enable_gpu_renders()
        render_dylan(
            output_path=sample_dir, 
            sample_id=sample_id, 
            garment_name=garment_name, 
            gender=gender, 
            fabric=fabric, 
            garment_verts=cloth_state["verts"], 
            garment_faces=cloth_state["faces"],
            garment_uv_verts=cloth_state["uv_verts"], 
            garment_uv_faces=cloth_state["uv_faces"], 
            garment_texture=garment_texture, 
            num_camera_angles=1, 
            camera_intrinsic=camera_intrinsic,
            render_animation=True,
            z_offset=-0.4
        )

            

Animating 00471_Tshirt_153, action sequence: 0
Animating 07135_Tshirt_4909, action sequence: 0
Animating 07516_Tshirt_2950, action sequence: 0
Animating 04199_Tshirt_3855, action sequence: 0
Animating 04956_Tshirt_2092, action sequence: 0
Animating 06698_Tshirt_321, action sequence: 0
Animating 01841_Tshirt_2878, action sequence: 0
Animating 05440_Tshirt_1903, action sequence: 0
Animating 03685_Tshirt_3008, action sequence: 0
Animating 06305_Tshirt_44, action sequence: 0
Animating 05739_Tshirt_174, action sequence: 0
Animating 04571_Tshirt_4765, action sequence: 0
Animating 04325_Tshirt_3309, action sequence: 0
Animating 05839_Tshirt_3309, action sequence: 0
Animating 00387_Tshirt_2659, action sequence: 0
Animating 03965_Tshirt_3646, action sequence: 0
Animating 07516_Tshirt_3262, action sequence: 0
Animating 04891_Tshirt_4354, action sequence: 0
Animating 05542_Tshirt_3908, action sequence: 0
Animating 06105_Tshirt_80, action sequence: 0
Animating 05739_Tshirt_3482, action sequence: 0